<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=eea544415ffa585dbc8a1d4d973146c11517943773676264fd0e3095a91c24fb
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-10 13:17:42
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  57.44 K
Current:  1.42 C
-------------------
Today PnL: 57.66 K (0.41%)
Current PnL: -19.83 L (-13.21%)
CY Booked + Current PnL: -8.39 L (-5.59%)
-------------------
Total profit:  1.82 L
Total loss:  -21.66 L
-------------------
Total Booked + Current PnL: 18.59 L (15.09%)
Total Booked PnL: 38.42 L (31.2%)
Curr Year Booked PnL: 11.44 L (8.07%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 87.34 L (61.62%)
Deployed:  1.23 C
Current:  1.42 C
CAGR/XIRR %: 8.48%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-40.35,65.0,H-MC,14.27,118921.0,18417.0,10691.0,-2.25,18.32,8.99,28.96,79.0,1.72,0.84,58.52,MH,ATH,METALS
19,CIPLA,1662.74,-19.78,58.0,X-LC,3.67,213610.0,9110.0,14184.0,3.05,4.45,6.64,11.39,10.0,0.64,1.51,13.81,X40N,ATH,PHARMA
14,BLUESTARCO,2080.00,11.77,55.0,X-MC,3.17,192940.0,28270.0,15069.0,1.96,17.17,7.81,26.31,89.0,1.88,1.37,26.52,X40N,NTT,AC
84,VOLTAS,1530.00,1.72,63.0,X-MC,1.10,213435.0,21693.0,16072.0,2.96,11.31,7.53,19.69,99.0,1.35,1.51,18.88,XY25,NTT,AC
77,TTKPRESTIG,770.00,93.38,40.0,M-SC,1.00,83316.0,-17461.0,17555.0,-0.16,-17.33,21.07,0.09,245.0,-0.99,0.59,9.46,OX40N,NTT,DURABLES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,CIPLA,1662.74,-19.78,58.0,X-LC,3.67,213610.0,9110.0,14184.0,3.05,4.45,6.64,11.39,10.0,0.64,1.51,13.81,X40N,ATH,PHARMA
21,COFFEEDAY,80.00,-43.66,56.0,L-SC,29.30,84480.0,-29069.0,69122.0,3.04,-25.60,81.82,35.27,268.0,-0.42,0.60,106.77,XR,NTT,HOTELS
84,VOLTAS,1530.00,1.72,63.0,X-MC,1.10,213435.0,21693.0,16072.0,2.96,11.31,7.53,19.69,99.0,1.35,1.51,18.88,XY25,NTT,AC
46,KPIGREEN,731.05,-8.25,35.0,H-SC,6.28,109406.0,-15891.0,74823.0,2.93,-12.68,68.39,47.03,141.0,-0.21,0.77,36.99,MH,ATH,POWER
53,QUESS,986.00,-46.01,36.0,X-SC,37.76,53611.0,-11395.0,162323.0,2.86,-17.53,302.78,232.18,198.0,-0.07,0.38,2.86,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HINDZINC,730.22,32.35,63.0,M-LC,7.77,215673.0,10597.0,101970.0,-3.21,5.17,47.28,54.89,52.0,0.10,1.53,30.51,X5K,ATH,METALS
60,SAIL,228.00,44.35,51.0,M-MC,12.68,228131.0,3169.0,164482.0,-2.92,1.41,72.10,74.53,192.0,0.02,1.62,34.12,XY24,BTT,STEEL
72,TATAELXSI,9161.00,-23.71,47.0,H-MC,6.63,103151.0,-28684.0,70906.0,-2.58,-21.76,68.74,32.03,98.0,-0.40,0.73,15.54,OX40N,NTT,IT
50,NATIONALUM,244.55,-40.35,65.0,H-MC,14.27,118921.0,18417.0,10691.0,-2.25,18.32,8.99,28.96,79.0,1.72,0.84,58.52,MH,ATH,METALS
0,5PAISA,593.00,106.73,31.0,H-SC,7.82,140566.0,-77930.0,106127.0,-2.13,-35.67,75.50,12.90,153.0,-0.73,1.00,7.00,OX40N,NTT,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,108.87,64.0,H-SC,13.80,131922.0,-10557.0,30078.0,1.35,-7.41,22.80,13.70,163.0,-0.35,0.93,55.70,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.38,40.0,M-SC,1.00,83316.0,-17461.0,17555.0,-0.16,-17.33,21.07,0.09,245.0,-0.99,0.59,9.46,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.79,50.0,H-SC,1.94,222039.0,-47628.0,83953.0,0.55,-17.66,37.81,13.47,138.0,-0.57,1.57,12.90,XY24,NTT,PAINTS
72,TATAELXSI,9161.0,-23.71,47.0,H-MC,6.63,103151.0,-28684.0,70906.0,-2.58,-21.76,68.74,32.03,98.0,-0.40,0.73,15.54,OX40N,NTT,IT
18,CERA,9475.0,-24.58,33.0,H-SC,1.81,137356.0,-38547.0,80573.0,-0.85,-21.91,58.66,23.89,149.0,-0.48,0.97,18.63,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,44.35,51.0,M-MC,12.68,228131.0,3169.0,164482.0,-2.92,1.41,72.10,74.53,192.0,0.02,1.62,34.12,XY24,BTT,STEEL
32,HINDZINC,730.22,32.35,63.0,M-LC,7.77,215673.0,10597.0,101970.0,-3.21,5.17,47.28,54.89,52.0,0.10,1.53,30.51,X5K,ATH,METALS
79,UNIONBANK,163.00,-11.63,58.0,M-LC,9.19,158555.0,17715.0,26780.0,1.18,12.58,16.89,31.59,66.0,0.66,1.12,42.40,XY24,NTT,BANKS
86,WIPRO,420.00,-11.98,56.0,M-LC,5.70,154721.0,3776.0,105674.0,1.28,2.50,68.30,72.51,53.0,0.04,1.10,8.67,XR,NTT,IT
11,BANKINDIA,190.00,-24.22,69.0,H-MC,11.96,195572.0,15764.0,96652.0,1.09,8.77,49.42,62.52,88.0,0.16,1.39,45.84,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-40.35,65.0,H-MC,14.27,118921.0,18417.0,10691.0,-2.25,18.32,8.99,28.96,79.0,1.72,0.84,58.52,MH,ATH,METALS
11,BANKINDIA,190.00,-24.22,69.0,H-MC,11.96,195572.0,15764.0,96652.0,1.09,8.77,49.42,62.52,88.0,0.16,1.39,45.84,XR,NTT,BANKS
32,HINDZINC,730.22,32.35,63.0,M-LC,7.77,215673.0,10597.0,101970.0,-3.21,5.17,47.28,54.89,52.0,0.10,1.53,30.51,X5K,ATH,METALS
86,WIPRO,420.00,-11.98,56.0,M-LC,5.70,154721.0,3776.0,105674.0,1.28,2.50,68.30,72.51,53.0,0.04,1.10,8.67,XR,NTT,IT
37,INDIAMART,4810.62,-54.45,43.0,H-SC,8.74,127359.0,4023.0,127601.0,-0.38,3.26,100.19,106.72,119.0,0.03,0.90,27.40,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-18.16,36.0,X-LC,5.55,206694.0,-45074.0,98283.0,0.53,-17.90,47.55,21.13,27.0,-0.46,1.46,11.18,X40,ATH,PAINTS
23,DABUR,735.00,-12.12,36.0,X-MC,3.75,237650.0,-10418.0,118825.0,0.95,-4.20,50.00,43.70,102.0,-0.09,1.68,9.64,XY24,BTT,FMCG
53,QUESS,986.00,-46.01,36.0,X-SC,37.76,53611.0,-11395.0,162323.0,2.86,-17.53,302.78,232.18,198.0,-0.07,0.38,2.86,XY24,NTT,MISC
22,COLPAL,3767.14,-4.98,37.0,X-MC,7.28,221710.0,-41655.0,154997.0,0.38,-15.82,69.91,43.04,84.0,-0.27,1.57,2.93,XY25,ATH,FMCG
56,RELAXO,1176.00,-40.07,38.0,X-SC,2.56,81630.0,-63530.0,139457.0,-0.84,-43.77,170.84,52.31,136.0,-0.46,0.58,9.27,X40N,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,HAVELLS,2069.16,-4.48,47.0,X-MC,0.50,236636.0,-11095.0,86159.0,2.09,-4.48,36.41,30.30,92.0,-0.13,1.68,8.22,X40,ATH,ELECTRICAL
84,VOLTAS,1530.00,1.72,63.0,X-MC,1.10,213435.0,21693.0,16072.0,2.96,11.31,7.53,19.69,99.0,1.35,1.51,18.88,XY25,NTT,AC
73,TATAMOTORS,1065.00,-22.16,44.0,X-LC,1.59,246840.0,-29683.0,139761.0,-0.16,-10.73,56.62,39.81,54.0,-0.21,1.75,18.30,XY24,NTT,AUTO
65,SIEMENS,4671.50,-1.54,48.0,H-LC,1.67,158170.0,-27925.0,75400.0,-0.47,-15.01,47.67,25.51,15.0,-0.37,1.12,16.56,AR,ATH,ELECTRICAL
34,ICICIGI,2260.25,-19.41,45.0,X-MC,1.82,138424.0,2291.0,28834.0,-0.64,1.68,20.83,22.86,91.0,0.08,0.98,14.51,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-20.80,41.0,X-LC,10.33,285282.0,-30560.0,143212.0,0.28,-9.68,50.20,35.67,5.0,-0.21,2.02,2.84,X40N,ATH,FMCG
53,QUESS,986.00,-46.01,36.0,X-SC,37.76,53611.0,-11395.0,162323.0,2.86,-17.53,302.78,232.18,198.0,-0.07,0.38,2.86,XY24,NTT,MISC
22,COLPAL,3767.14,-4.98,37.0,X-MC,7.28,221710.0,-41655.0,154997.0,0.38,-15.82,69.91,43.04,84.0,-0.27,1.57,2.93,XY25,ATH,FMCG
42,ITC,452.00,-39.19,45.0,X-LC,2.41,196554.0,-3584.0,24471.0,0.51,-1.79,12.45,10.44,4.0,-0.15,1.39,3.67,X40,NTT,FMCG
9,BAJAJHFL,181.50,-14.48,38.0,X-MC,4.96,180334.0,-20172.0,117325.0,-0.12,-10.06,65.06,48.45,90.0,-0.17,1.28,3.99,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-46.01,36.0,X-SC,37.76,53611.0,-11395.0,162323.0,2.86,-17.53,302.78,232.18,198.0,-0.07,0.38,2.86,XY24,NTT,MISC
56,RELAXO,1176.00,-40.07,38.0,X-SC,2.56,81630.0,-63530.0,139457.0,-0.84,-43.77,170.84,52.31,136.0,-0.46,0.58,9.27,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-34.87,43.0,X-SC,4.42,97376.0,-31294.0,76596.0,0.08,-24.32,78.66,35.20,219.0,-0.41,0.69,11.77,X40,NTT,FOOTWEAR
17,CAMS,5211.76,-4.72,51.0,X-SC,1.44,108248.0,6244.0,37681.0,0.91,6.12,34.81,43.06,122.0,0.17,0.77,24.29,X40N,ATH,MISC
33,HONAUT,58357.33,-22.39,48.0,X-SC,2.13,109320.0,-18618.0,65756.0,0.55,-14.55,60.15,36.84,143.0,-0.28,0.77,11.92,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-28.36,51.0,X-LC,12.16,284914.0,-61042.0,129294.0,-1.00,-17.64,45.38,19.73,1.0,-0.47,2.02,4.94,X40,ATH,IT
40,INFY,2275.00,-17.44,57.0,X-LC,7.78,321950.0,9062.0,162617.0,0.15,2.90,50.51,54.87,3.0,0.06,2.28,9.74,X40,BTT,IT
42,ITC,452.00,-39.19,45.0,X-LC,2.41,196554.0,-3584.0,24471.0,0.51,-1.79,12.45,10.44,4.0,-0.15,1.39,3.67,X40,NTT,FMCG
83,VBL,671.64,-20.80,41.0,X-LC,10.33,285282.0,-30560.0,143212.0,0.28,-9.68,50.20,35.67,5.0,-0.21,2.02,2.84,X40N,ATH,FMCG
1,ABB,7934.00,-40.45,46.0,H-LC,7.50,248760.0,-12859.0,132067.0,0.70,-4.92,53.09,45.57,7.0,-0.10,1.76,5.85,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-43.66,56.0,L-SC,29.30,84480.0,-29069.0,69122.0,3.04,-25.60,81.82,35.27,268.0,-0.42,0.60,106.77,XR,NTT,HOTELS
49,MASFIN,398.61,-20.05,43.0,H-SC,10.38,90945.0,-7035.0,28639.0,0.65,-7.18,31.49,22.05,152.0,-0.25,0.64,32.20,XR,ATH,FINANCE
50,NATIONALUM,244.55,-40.35,65.0,H-MC,14.27,118921.0,18417.0,10691.0,-2.25,18.32,8.99,28.96,79.0,1.72,0.84,58.52,MH,ATH,METALS
78,UJJIVANSFB,60.00,108.87,64.0,H-SC,13.80,131922.0,-10557.0,30078.0,1.35,-7.41,22.80,13.70,163.0,-0.35,0.93,55.70,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-185.23,67.0,M-SC,15.45,147555.0,-1665.0,145843.0,1.79,-1.12,98.84,96.62,208.0,-0.01,1.05,59.69,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-185.23,67.0,M-SC,15.45,147555.0,-1665.0,145843.0,1.79,-1.12,98.84,96.62,208.0,-0.01,1.05,59.69,XY25,NTT,FINANCE
69,SURYODAY,240.00,68.86,67.0,H-SC,9.07,154367.0,-24704.0,90428.0,1.84,-13.80,58.58,36.71,135.0,-0.27,1.09,53.16,XR,NTT,BANKS
32,HINDZINC,730.22,32.35,63.0,M-LC,7.77,215673.0,10597.0,101970.0,-3.21,5.17,47.28,54.89,52.0,0.10,1.53,30.51,X5K,ATH,METALS
26,GREENPANEL,537.00,234.63,60.0,M-SC,4.69,149426.0,-33652.0,112637.0,0.38,-18.38,75.38,43.14,230.0,-0.30,1.06,40.56,XY24,NTT,MISC
68,STARHEALTH,761.00,17.10,68.0,H-SC,7.42,260576.0,-20547.0,153401.0,0.08,-7.31,58.87,47.26,144.0,-0.13,1.85,39.61,XY24,NTT,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.15
1,25,43.45
2,50,75.19


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.97
LC    32.30
MC    23.74
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.91
X40      14.62
X40N     11.27
XR       11.02
XY25     10.44
AR        8.15
OX40N     7.60
X200      1.75
MH        1.61
X5K       1.53
SR        1.11
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.46
X-LC    22.91
X-MC    17.20
M-SC    12.52
H-LC     4.63
H-MC     4.61
X-SC     4.53
M-LC     3.75
M-MC     1.62
L-SC     1.46
L-LC     1.01
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.74,-5.77,40.63
IT,12.86,-15.92,78.70
FINANCE,10.50,-13.93,68.95
MISC,6.91,-17.82,83.10
BANKS,6.42,-10.04,67.93
PAINTS,5.48,-19.09,36.85
ELECTRICAL,5.33,-9.36,47.73
INSURANCE,3.79,-3.44,42.10
AC,3.57,3.82,12.29


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3211603.0,22
XR,1355216.0,14
AR,1177690.0,9
X40,775772.0,10
X40N,645256.0,9
OX40N,573748.0,10
XY25,463820.0,7
SR,265058.0,2
X5K,101970.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3403102.0,25
M-SC,1276284.0,15
X-MC,1106149.0,12
X-LC,1006266.0,13
X-SC,541677.0,6
H-MC,358958.0,4
H-LC,286059.0,3
L-SC,256093.0,3
M-LC,234424.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1193997.0      6
           AR         806079.0      5
           XR         795188.0      7
M-SC       XY24       688927.0      6
X-MC       XY24       588401.0      4
X-LC       X40        484891.0      5
X-SC       X40N       302758.0      4
H-SC       OX40N      267957.0      4
           SR         265058.0      2
X-MC       X40        214285.0      4
X-LC       X40N       210104.0      3
H-LC       AR         207467.0      2
X-LC       XY24       205984.0      2
H-MC       XY24       180709.0      1
X-MC       XY25       171069.0      2
L-SC       XR         167643.0      2
M-MC       XY24       164482.0      1
M-SC       AR         164144.0      2
X-SC       XY24       162323.0      1
M-SC       OX40N      146435.0      4
           XY25       145843.0      1
X-MC       X40N       132394.0      2
M-SC       XR         130935.0      2
M-LC       XR         105674.0      1
X-LC       XY25       105287.0      3
M-LC       X5K        101970.0      1
H-MC       XR          96652.0      1
L-SC       OX40N       88450.0      1
H-LC       X200        78592.0      1
X-SC       X40         76596.0      1
H-SC       MH          74823.0      1
H-MC       OX40N       70906.0      1
L-MC       XR          59124.0      1
L-LC       XY25        41621.0      1
M-LC       XY24        26780.0      1
H-MC       MH          10691.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
